<a href="https://colab.research.google.com/github/colinder/Naver_MovieReview_LSTM/blob/main/Naver_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naver 영화 리뷰 감정분석**
데이터 셋은 https://github.com/e9t/nsmc 의 자료를 사용


In [22]:
import urllib.request
import pandas as pd

## data 가져오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="total_data.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

total_data = pd.read_table("total_data.txt")
x_train = pd.read_table("ratings_train.txt")
x_text = pd.read_table("ratings_test.txt")

print("총 데이터 개수: ", len(total_data))
print("훈련 데이터 개수:", len(x_train))
print("검증 데이터 개수:", len(x_text))

총 데이터 개수:  200000
훈련 데이터 개수: 150000
검증 데이터 개수: 50000


# *불러온 데이터 __확인__

In [23]:
x_train[:10]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


# 정규표현식을 사용해 '!'나'.' 등 기호를 삭제
 

In [27]:
x_train['document'] = x_train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣. ]","")

In [28]:
## 정규표현식 적용 확인
x_train[:10]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
5,5403919,막 걸음마 뗀 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나,1


# 중복 데이터 확인

In [29]:
## 데이터 정제
# 중복검사
print("중복을 제거한 데이터의 수: ",x_train["document"].nunique())

## document열 기준 중복 데이터 제거 inplace=True (x_train을 변경)
x_train.drop_duplicates(subset=['document'], inplace=True)

print("중복제거 진행 후 데이터 수: ", len(x_train))

중복을 제거한 데이터의 수:  143682
중복제거 진행 후 데이터 수:  143683


In [30]:
## Null값 판단 및 제거

IsNull_indexs = []
for i, v in enumerate(x_train["document"].isnull()):
  if v == True:
    IsNull_indexs.append(i)

print("null값 갯수:", len(IsNull_indexs), "/", 'null값 index', IsNull_indexs, )


null값 갯수: 1 / null값 index [25190]


In [31]:
## null값 확인
x_train[25188:25193]

,id,document,label
25855,9284537,시간이 아깝다는 생각밖에 안드는군,0
25856,9475362,샘 킴 반지좀빼고요리하지 양손어다반지 비위생,0
25857,2172111,NaN,1
25858,8475146,진짜재밋어요 또개봉했으면 좋겠다 또봐야지,1
25859,7783279,벌써 년 이나 흘러버렸네 추억은 추억일때 아름다운 법 년 이상 영화는 이제 안볼란다...,0


In [32]:
# 어떤 행이던지 null 값이 있다면 제거 
x_train = x_train.dropna(how='any')

print("null값 제거 후 데이터 수 :", len(x_train))

null값 제거 후 데이터 수 : 143682


In [33]:
## null값 삭제 확인
x_train[25188:25193]

,id,document,label
25855,9284537,시간이 아깝다는 생각밖에 안드는군,0
25856,9475362,샘 킴 반지좀빼고요리하지 양손어다반지 비위생,0
25858,8475146,진짜재밋어요 또개봉했으면 좋겠다 또봐야지,1
25859,7783279,벌써 년 이나 흘러버렸네 추억은 추억일때 아름다운 법 년 이상 영화는 이제 안볼란다...,0
25860,2861998,보기 전부터 스토리는 기대도 안했고 선수라도 보는 재미를 원했건만,0


# *데이터 토큰화(벡터화) 진행

토큰화는 Kkma, Komoran 두가지로 테스트 예정</br>
둘 중 프로젝트에 더 적합한 모습을 보이는 라이브러리로 진행을 위한 테스트


In [ ]:
!pip install konlpy

from konlpy.tag import Kkma, Komoran


